In [55]:
import pandas as pd
import numpy as np
import PivotTable as pt
import process_mangement as pm
import os

In [56]:
# 실제 코드가 완성된 후에 앞부분은 SQLite에서 QUERY를 통해 가져와서 바로 DATAFRAME으로 변환하는 방식으로 변경할 예정
# 지금은 테스트를 위해 CSV 파일을 사용

# 데이터 불러오기
base_path_test = os.path.join(os.getcwd(), 'test_file')

df_working_f = pd.read_csv(os.path.join(base_path_test, '개별속성_작업파일(Test).csv'), dtype={'공정번호' : str}) # 작업파일
df_attrs_stdds = pd.read_csv(os.path.join(base_path_test, '개별속성_표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 표준데이터시트
df_attrs_nonstdds = pd.read_csv(os.path.join(base_path_test, '개별속성_비표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 비표준데이터시트
df_attrs_header = pd.read_csv(os.path.join(base_path_test, '속성해더.csv'), dtype={'공정번호' : str}) # 속성해더

In [57]:
# df_working 칼럼값 수정
# 표준데이터시트에서 추출된 태그와 비표준데이터시트에서 추출된 태그가 대표 태그로 올라와 있을 경우 [표준데이터시트] 칼럼과 [선작업데이터] 칼럼에 값이 들어가 있지 않으므로, 이를 채워줘야 함 

df_working_f['표준데이터시트'] = df_working_f.apply(lambda x: x['SRNo'] if x['표준데이터시트'] !=np.nan and x['출처'] == "표준" else x['표준데이터시트'], axis=1)
df_working_f['선작업데이터'] = df_working_f.apply(lambda x: x['SRNo'] if x['선작업데이터'] !=np.nan and x['출처'] == "비표준" else x['선작업데이터'], axis=1)
df_working_f

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,공종별 분류 코드,속성 그룹 코드,속성1,속성2,속성3,속성4,속성5,속성6,속성7
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00001,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AE00004,NaN,PID,07,LS0002,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AE00005,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,11001,01_속성명,서비스,타입,측정범위,제조사,정비주기,NaN,NaN
7,AE0006,NaN,PID,07,NaN,HH0003,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AE0007,NaN,PID,07,NaN,HH0004,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AE0008,NaN,PID,07,NaN,NaN,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# 표준데이터시트와 비표준데이터시트 데이터프레임에 대표 SRNo를 추가
## df_working_f의 표준데이터시트 칼럼란이 df_attrs_std에 있는 SRNo와 일치하는 경우 df_working_f의 해당 레코드의 SRNo를 df_attrs_std의 대표 SRNo로 하며, 일치하지 않는 경우에는 np.nan으로 처리
## 비표준데이터시트도 같은 방식으로 하되 기준이 되는 칼럼은 df_wokring_f의 선작업데이터 칼럼

def get_representative_srno(df_base, df_right, lookup_columns) :
    """개별속성 입력 데이터에 df_working_f에서의 대표 SRNo를 추가한다"""
    df_right_join = df_working_f[['SRNo', lookup_columns]]
    df_merge = pd.merge(df_base, df_right_join, how='left', left_on='SRNo', right_on=lookup_columns)
    df_merge.drop(lookup_columns, axis=1, inplace=True)
    df_merge.rename(columns={'SRNo_x' : 'SRNo', 'SRNo_y':'대표 SRNo'}, inplace=True)
    return df_merge

df_attrs_stdds = get_representative_srno(df_attrs_stdds, df_working_f, '표준데이터시트')
df_attrs_stdds.dropna(subset=['대표 SRNo'], inplace=True)
df_attrs_nonstdds = get_representative_srno(df_attrs_nonstdds, df_working_f, '선작업데이터')
df_attrs_nonstdds.dropna(subset=['대표 SRNo'], inplace=True)

In [58]:
# 공종별 분류 코드 중 하나만 선택. 실제 코드 구현에서는 for loop를 사용하여 모든 코드를 가져와야 함(속성그룹 코드 '01_속성명'으로 필터링)

type_code = df_attrs_header['공종별 분류 코드'].iloc[0]

# df_working_filtered

df_working_filtered = df_working_f[df_working_f['속성 그룹 코드'] == '03_DATA']
df_working_filtered = df_working_filtered[df_working_filtered['공종별 분류 코드'] == type_code]

# 속성해더의 행을 이용한 새 데이터프레임 생성
attrs_columns = df_attrs_header.iloc[0].to_list()
df_cct = pd.DataFrame(columns=attrs_columns)

# 필터링한 데이터와 df_working_filtered를 결합하여 내용이 채워진 df_cct를 만듬
df_cct

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유


In [60]:
# 개별속성 테이블 피벗 준비

df_attrs_stdds_filtered = df_attrs_stdds[['대표 SRNo', '속성명', '속성값']]
df_attrs_stdds_filtered.rename(columns={'대표 SRNo' : 'SRNo'}, inplace=True)

df_attrs_nonstdds_filtered = df_attrs_nonstdds[['대표 SRNo', '속성명', '속성값']]
df_attrs_nonstdds_filtered.rename(columns={'대표 SRNo' : 'SRNo'}, inplace=True)

C:\Users\ggaps\AppData\Local\Temp\ipykernel_8368\1028042101.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attrs_stdds_filtered.rename(columns={'대표 SRNo' : 'SRNo'}, inplace=True)
C:\Users\ggaps\AppData\Local\Temp\ipykernel_8368\1028042101.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attrs_nonstdds_filtered.rename(columns={'대표 SRNo' : 'SRNo'}, inplace=True)


In [61]:
# 개별속성 케이블 피벗팅

tb = pt.Table(df_attrs_stdds_filtered)
df_tb = tb.convert_pivot()
df_tb

속성명,SRNo,방폭,부식여유,비파괴,서비스,압력,온도,유량,정비주기,제조사,측정범위,크기,타입
0,AE00002,유,3|mm,No,NaN,NaN,AMB,20|m^3/h,NaN,NaN,NaN,2000,NaN
1,AE0006,NaN,NaN,NaN,Tk-503,NaN,NaN,NaN,1년,영월압력계,range 0 ~ 50,NaN,Bourdon
2,AE0007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bourdon
3,AE0010,NaN,NaN,No,NaN,1.5|kgf/cm^2,15|degC,NaN,NaN,놀이공원안전사고현황,NaN,NaN,의자사용방법
4,HH0002,Yes,NaN,NaN,NaN,F.V,45,NaN,NaN,NaN,NaN,1000,NaN


In [62]:
# 빈 df_cct 칼럼과 합침

df_cct_result = pd.concat([df_cct, df_tb], axis=0)
df_cct_result

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유,서비스,정비주기,제조사,측정범위,타입
0,AE00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMB,NaN,20|m^3/h,유,2000,No,3|mm,NaN,NaN,NaN,NaN,NaN
1,AE0006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tk-503,1년,영월압력계,range 0 ~ 50,Bourdon
2,AE0007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bourdon
3,AE0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15|degC,1.5|kgf/cm^2,NaN,NaN,NaN,No,NaN,NaN,NaN,놀이공원안전사고현황,NaN,의자사용방법
4,HH0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,F.V,NaN,Yes,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
common_columns = df_cct_result.columns.intersection(df_attrs_header.columns)
df_cct_result = pd.merge(df_cct_result, df_working_filtered[common_columns], how='left', on='SRNo')
for col in df_cct_result.columns :
    if '_x' in str(col) :
        df_cct_result.drop(col, axis=1, inplace=True)
    elif '_y' in str(col) :
        df_cct_result.rename(columns={col : col.replace('_y', '')}, inplace=True)
df_cct_result = df_cct_result[df_cct.columns.to_list()]
df_cct_result

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
0,AE00002,NaN,PID,07,LS0001,HH0001,NaN,NaN,AMB,NaN,20|m^3/h,유,2000,No,3|mm
1,AE0006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AE0007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AE0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15|degC,1.5|kgf/cm^2,NaN,NaN,NaN,No,NaN
4,HH0002,NaN,표준,07,NaN,HH0002,NaN,NaN,45,F.V,NaN,Yes,1000,NaN,NaN


In [64]:
# 공종별 분류 코드 추가(추후 실제 코드에서 로직 추가)

df_cct_result[type_code] = type_code
df_cct_result['01_속성명'] = '03_DATA'
df_cct_result

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
0,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,AMB,NaN,20|m^3/h,유,2000,No,3|mm
1,AE0006,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AE0007,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AE0010,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,15|degC,1.5|kgf/cm^2,NaN,NaN,NaN,No,NaN
4,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,45,F.V,NaN,Yes,1000,NaN,NaN


In [65]:
# 해더를 데이트프레임이 첫 행을 만들기

def make_header_to_firts_row(df) :
    header_df = pd.DataFrame([df.columns.tolist()], columns=df.columns)
    new_df = pd.concat([header_df, df], ignore_index=True)
    return new_df

In [66]:
df_cct_result = make_header_to_firts_row(df_cct_result)
df_cct_result

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,AMB,NaN,20|m^3/h,유,2000,No,3|mm
2,AE0006,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AE0007,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AE0010,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,15|degC,1.5|kgf/cm^2,NaN,NaN,NaN,No,NaN
5,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,45,F.V,NaN,Yes,1000,NaN,NaN


In [67]:
# 해더 길이 자르기
def cut_header(df, df_cut) :
    main_header = df.columns.to_list()
    main_header = main_header[: len(df_cut.columns.to_list())]
    return main_header

# 메인 해더로 해더 이름 바꾸기
def change_header(df, main_header) :
    df.columns = main_header
    return df

# main_header = df_working_filtered.columns.to_list()
# main_header = main_header[: len(df_cct_result.columns.to_list())]
# main_header

In [68]:
main_header = cut_header(df_working_filtered, df_cct_result)
main_header

['SRNo',
 '작업자',
 '출처',
 '공정번호',
 '선작업데이터',
 '표준데이터시트',
 '공종별 분류 코드',
 '속성 그룹 코드',
 '속성1',
 '속성2',
 '속성3',
 '속성4',
 '속성5',
 '속성6',
 '속성7']

In [69]:
change_header(df_cct_result, main_header)

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,공종별 분류 코드,속성 그룹 코드,속성1,속성2,속성3,속성4,속성5,속성6,속성7
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,AMB,NaN,20|m^3/h,유,2000,No,3|mm
2,AE0006,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AE0007,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AE0010,NaN,NaN,NaN,NaN,NaN,10001,03_DATA,15|degC,1.5|kgf/cm^2,NaN,NaN,NaN,No,NaN
5,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,45,F.V,NaN,Yes,1000,NaN,NaN
